In [124]:
import networkx as nx

In [ ]:
class Top:
    ''' A class for representing a finite topological space. 
        ------------
        Attributes |
        ----------------------------------------------------------------
        points: a set {x | x in X } of the points of the top. space
                The elements should all be hashable

        opens: a dictionary of the basic open sets. It is of the form
        { x : { y | x <= y} } where x <= y means x in closure({y})

        closures:   dictionary of the closures of each point
                    { x : cl({x}) }
                    Starts as empty because computing all is O(n^2)
                    Run getClosures() method if all are needed. Else,
                    just call X.cl(x) or X[x] for closure of 1 point
        ----------
        Methods  |
        ----------------------------------------------------------------
        


    
    '''
    def __init__(self, data):
        # can pass a networkx graph as the data
        if type(data) == nx.DiGraph:
            opens = {}
            for node in data.nodes:
                opens[node] = nx.descendants(data,node) | {node}
            self.opens = opens
            self.points = set(data.nodes)
            self.closures = {}
        elif type(data) == dict:
            self.opens = data
            self.points = set(data)
            self.closures = {}

    def __repr__(self):
        return str(self.points)
    
    def __len__(self):
        return len(self.points)
    
    def __contains__(self, x):
        return x in self.points
    
    def __iter__(self):
        return iter(self.points)

    def cl(self, x):
        # returns the closure of the singleton {x}
        assert x in self.points,\
        f'{x} is not a point in the topological space'
        open = set()
        for point in self.points:
            if x not in self.opens[point]:
                open |= self.opens[point] 
        # open = largest open set not containing x
        # closure of {x} is its complement
        return self.points - open
    
    def __call__(self, x):
        # Top(x) returns the same as self.opens[x]
        return self.opens[x]
        
    def __getitem__(self, x):
        # Top[x] returns the same as Top.cl(x) or Top.closures[x]
        # This convention mirrors the notation (a,b) for open interval 
        # and [a,b] for closed interval
        try:
            return self.closures[x]
        except:
            self.closures[x] = self.cl(x)
            return self.closures[x]
        
    def getClosures(self):
        # populates Top.closures for all elements
        for x in self:
            try: # don't bother if it's already computed
                self.closures[x]
            except:
                self.closures[x] = self.cl(x)

    def isleq(self, x, y):
        # partial order defined by x<=y iff x in cl(y)
        return x in self[y]
    
    def isT0(self):
        # True iff self.opens[x] = self.opens[y] implies x = y
        checked = set()
        for x in self:
            for y in self.points - (checked | {x}):
                if self(x) == self(y):
                    return False
            checked |= {x} # don't check U_x=U_y and U_y=U_x separately
        return True
    
    def isT1(self):
        # for finite spaces, this is the same as being discrete
        for x in self:
            if self(x) != {x}:
                return False
        return True
    
    def __mul__(self, other):
        return self.product(other)
    
    def product(self, other):
        # cartesian product
        def prod(set1, set2):
            return {(x, y) for x in set1 for y in set2}
        points = prod(self.points, other.points)
        opens = {point : prod(self(point[0]), other(point[1])) \
                 for point in points}
        return Top(opens)
    







In [129]:
X = Top({0:{0,1,3}, 1:{1}, 2:{1,2,3}, 3:{3}})
X.getClosures()
X.closures

{0: {0}, 1: {0, 1, 2}, 2: {2}, 3: {0, 2, 3}}

In [130]:
X*X

{(0, 1), (1, 2), (2, 1), (3, 1), (0, 2), (2, 2), (1, 0), (3, 2), (1, 3), (0, 0), (1, 1), (0, 3), (2, 0), (3, 0), (2, 3), (3, 3)}